In [11]:
# Import Libraries
import pandas as pd
import numpy as np
import psycopg2
import sqlite3
import config
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, func, inspect, Column, Integer, String, Float
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

Base = declarative_base()

In [12]:
# Read in the data
covid_data_df = pd.read_csv('covid-data.csv')
covid_data_df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2/24/20,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2/25/20,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2/26/20,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2/27/20,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2/28/20,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN


In [13]:
# Data types for the columns

covid_data_df.dtypes

iso_code                                  object
continent                                 object
location                                  object
date                                      object
total_cases                              float64
new_cases                                float64
new_cases_smoothed                       float64
total_deaths                             float64
new_deaths                               float64
new_deaths_smoothed                      float64
total_cases_per_million                  float64
new_cases_per_million                    float64
new_cases_smoothed_per_million           float64
total_deaths_per_million                 float64
new_deaths_per_million                   float64
new_deaths_smoothed_per_million          float64
reproduction_rate                        float64
icu_patients                             float64
icu_patients_per_million                 float64
hosp_patients                            float64
hosp_patients_per_mi

In [14]:
# Select the columns to keep

covid_data_df2 = covid_data_df[["iso_code", "continent", "location", "date", "total_cases", "total_deaths", "total_vaccinations", "people_fully_vaccinated", "population"]]
covid_data_df2.head()

,iso_code,continent,location,date,total_cases,total_deaths,total_vaccinations,people_fully_vaccinated,population
0,AFG,Asia,Afghanistan,2/24/20,1.0,NaN,NaN,NaN,38928341.0
1,AFG,Asia,Afghanistan,2/25/20,1.0,NaN,NaN,NaN,38928341.0
2,AFG,Asia,Afghanistan,2/26/20,1.0,NaN,NaN,NaN,38928341.0
3,AFG,Asia,Afghanistan,2/27/20,1.0,NaN,NaN,NaN,38928341.0
4,AFG,Asia,Afghanistan,2/28/20,1.0,NaN,NaN,NaN,38928341.0


In [15]:
# Data Cleaning
covid_data_df2=covid_data_df2.fillna(0)
covid_data_df2.head()

,iso_code,continent,location,date,total_cases,total_deaths,total_vaccinations,people_fully_vaccinated,population
0,AFG,Asia,Afghanistan,2/24/20,1.0,0.0,0.0,0.0,38928341.0
1,AFG,Asia,Afghanistan,2/25/20,1.0,0.0,0.0,0.0,38928341.0
2,AFG,Asia,Afghanistan,2/26/20,1.0,0.0,0.0,0.0,38928341.0
3,AFG,Asia,Afghanistan,2/27/20,1.0,0.0,0.0,0.0,38928341.0
4,AFG,Asia,Afghanistan,2/28/20,1.0,0.0,0.0,0.0,38928341.0


In [16]:
# Check any missing data
for col in covid_data_df2.columns:
    missing_data = np.mean(covid_data_df2[col].isnull())
    print('{} - {}%'.format(col, missing_data))

iso_code - 0.0%
continent - 0.0%
location - 0.0%
date - 0.0%
total_cases - 0.0%
total_deaths - 0.0%
total_vaccinations - 0.0%
people_fully_vaccinated - 0.0%
population - 0.0%


In [17]:
#Renaming the columns

covid_data_df2 = covid_data_df2.rename(
    columns={"iso_code":"Country_Code", "location":"Country"})
covid_data_df2

,Country_Code,continent,Country,date,total_cases,total_deaths,total_vaccinations,people_fully_vaccinated,population
0,AFG,Asia,Afghanistan,2/24/20,1.0,0.0,0.0,0.0,38928341.0
1,AFG,Asia,Afghanistan,2/25/20,1.0,0.0,0.0,0.0,38928341.0
2,AFG,Asia,Afghanistan,2/26/20,1.0,0.0,0.0,0.0,38928341.0
3,AFG,Asia,Afghanistan,2/27/20,1.0,0.0,0.0,0.0,38928341.0
4,AFG,Asia,Afghanistan,2/28/20,1.0,0.0,0.0,0.0,38928341.0
...,...,...,...,...,...,...,...,...,...
106809,ZWE,Africa,Zimbabwe,7/29/21,105656.0,3421.0,2324034.0,730378.0,14862927.0
106810,ZWE,Africa,Zimbabwe,7/30/21,107490.0,3490.0,2375361.0,751487.0,14862927.0
106811,ZWE,Africa,Zimbabwe,7/31/21,108860.0,3532.0,2413509.0,767910.0,14862927.0
106812,ZWE,Africa,Zimbabwe,8/1/21,109546.0,3583.0,2433341.0,779229.0,14862927.0


In [18]:
covid_data_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106814 entries, 0 to 106813
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Country_Code             106814 non-null  object 
 1   continent                106814 non-null  object 
 2   Country                  106814 non-null  object 
 3   date                     106814 non-null  object 
 4   total_cases              106814 non-null  float64
 5   total_deaths             106814 non-null  float64
 6   total_vaccinations       106814 non-null  float64
 7   people_fully_vaccinated  106814 non-null  float64
 8   population               106814 non-null  float64
dtypes: float64(5), object(4)
memory usage: 7.3+ MB


In [19]:
finalcovid_data_df2 = covid_data_df2[["continent","Country","Country_Code","population","date","total_cases","total_deaths","total_vaccinations","people_fully_vaccinated"]]
finalcovid_data_df2.head()

,continent,Country,Country_Code,population,date,total_cases,total_deaths,total_vaccinations,people_fully_vaccinated
0,Asia,Afghanistan,AFG,38928341.0,2/24/20,1.0,0.0,0.0,0.0
1,Asia,Afghanistan,AFG,38928341.0,2/25/20,1.0,0.0,0.0,0.0
2,Asia,Afghanistan,AFG,38928341.0,2/26/20,1.0,0.0,0.0,0.0
3,Asia,Afghanistan,AFG,38928341.0,2/27/20,1.0,0.0,0.0,0.0
4,Asia,Afghanistan,AFG,38928341.0,2/28/20,1.0,0.0,0.0,0.0


In [17]:
# Push the remade DataFrame to a new CSV file
finalcovid_data_df2.to_csv("Final Covid data.csv")

### Creating class for SQL Database

In [20]:
class Covid_Data(Base):
    __tablename__ = "Covid_Data"
    ID = Column(Integer, primary_key=True)
    continent = Column(String(255))
    Country = Column(String(255))
    Country_Code = Column(Integer)
    population = Column(Integer)
    date = Column(Integer)
    total_cases = Column(String(255))
    total_deaths = Column(String(255))
    total_vaccinations = Column(String(255))
    people_fully_vaccinated = Column(Integer)

In [21]:
# Connect to local database
#Create engine connection
#engine = create_engine("sqlite:///covid.sqlite", echo=False)
engine = create_engine("sqlite:///covid.sqlite")
Base.metadata.create_all(engine)
session = Session(engine)
conn = engine.connect()

In [22]:
#Uploading covid data to sqlite database
finalcovid_data_df2.to_sql("Covid_Data", engine, if_exists="append", index=False)

In [23]:
# Test connection
con = sqlite3.connect("covid.sqlite")
query ="SELECT continent,Country,Country_Code,population,date,total_cases,total_deaths,total_vaccinations,people_fully_vaccinated from Covid_Data"
country_df = pd.read_sql_query(query, con)
country_df.head()

,continent,Country,Country_Code,population,date,total_cases,total_deaths,total_vaccinations,people_fully_vaccinated
0,Asia,Afghanistan,AFG,38928341,2/24/20,1.0,0.0,0.0,0
1,Asia,Afghanistan,AFG,38928341,2/25/20,1.0,0.0,0.0,0
2,Asia,Afghanistan,AFG,38928341,2/26/20,1.0,0.0,0.0,0
3,Asia,Afghanistan,AFG,38928341,2/27/20,1.0,0.0,0.0,0
4,Asia,Afghanistan,AFG,38928341,2/28/20,1.0,0.0,0.0,0


### Postgress

In [24]:
# Create sqlite engine, reflect tables

engine = create_engine("sqlite:///covid.sqlite", echo=False)


In [25]:
#Reflect Database into ORM classes
#Base = automap_base()
Base = declarative_base()

In [26]:
host = "localhost"
user = "postgres"
password = "personal"
port = 5432

url = 'postgresql+psycopg2://'+user+":"+ "postgres" + "@" + host + ":" + "5432" + "/"

database_name = "Corona"

engine = create_engine(url)
conn = engine.connect()
conn.execute("commit")
conn.execute ("DROP DATABASE IF EXISTS " + database_name)
conn.execute("commit")
conn.execute("CREATE DATABASE " + database_name)
conn.close()
engine.dispose()

In [27]:
#Use "" Around Covid_Data when querying from Postgres
database_url = url + database_name
Corona_engine = create_engine(database_url)
finalcovid_data_df2.to_sql('Covid_Data', Corona_engine, if_exists='replace', index=False)

In [28]:
#Create engine connection

engine = create_engine("sqlite:///covid.sqlite")
Base.metadata.create_all(engine)
session = Session(engine)
conn = engine.connect()